In [1]:
import pandas as pd
import folium
import numpy as np
import os
import googlemaps
import requests
from textwrap import shorten
from bs4 import BeautifulSoup
import re

In [2]:
df = pd.read_csv('SecurityZoneSales - Sheet1.csv',dtype='str')

In [3]:
df

,Date,Address,Price,Last Sale,Link
0,"May 28, 2025",142 Via Palma,$17M,"$5.4M, 2016",https://therealdeal.com/miami/2025/06/17/palm-...
1,"Apr 8, 2025",129 Woodbridge Road,$14M,"$2.2M, 2013",https://www.yahoo.com/lifestyle/house-trumps-m...
2,"Mar 12, 2025",153 Kings Road,$14M,"$2.6M, 2004",https://www.usatoday.com/story/money/personalf...
3,"Feb 21, 2025",168 Kings Road,$12M,"$14M, 2023",https://www.palmbeachdailynews.com/story/busin...
4,"Jan 31, 2025",1090 South Ocean Boulevard,$19.5M,"$5.6M, 2020",https://therealdeal.com/miami/2025/02/11/marat...
5,"Jan 10, 2025",120 Clarendon Avenue,$27.5M,"$22.7M, 2021",https://therealdeal.com/miami/2025/01/15/goya-...
6,"Dec 17, 2024",130 Algoma Road,$14.3M,"$4.1M, 2021",https://www.palmbeachdailynews.com/story/busin...


## Geocode

In [4]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

%store -r map_box_api_key

In [5]:
df['Full_Address'] = df['Address'] + ' Palm Beach FL'

In [6]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Full_Address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [8]:
df['Link'] = 'Click here for story'

df["short_description"] = df["Link"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['Link'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["Link"] = '<a href="' + df["Link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]

In [9]:
from textwrap import shorten
import pandas as pd
import folium

# 1) Prepare your link column
df['short_last_sale'] = df['Last Sale'].apply(
    lambda u: shorten(u, width=20, placeholder="…")
)
df['link_html'] = df.apply(
    lambda row: (
        f'<a href="{row["Last Sale"]}" '
        'target="_blank" rel="noopener noreferrer">'
        f'{row["short_last_sale"]}</a>'
    ),
    axis=1
)

# 2) Tooltip helper
TOOLTIP_FIELDS = ['Date', 'Address', 'Price']

def tooltip_html(idx, max_chars=30):
    parts = ['<div class="popup-content">']
    for col in TOOLTIP_FIELDS:
        val = str(df.at[idx, col]).strip()
        if pd.notnull(val) and val:
            if col == 'Address' and len(val) > max_chars:
                val = '<br>'.join(
                    val[i:i+max_chars] for i in range(0, len(val), max_chars)
                )
            parts.append(f'<strong>{col}:</strong> {val}<br>')
    parts.append(f'<strong>Last Sale:</strong> {df.at[idx, "link_html"]}')
    parts.append('</div>')
    return ''.join(parts)

# 3) Initialize the map without default tiles
m = folium.Map(
    location=[df['lat'].mean(), df['lon'].mean()],
    zoom_start=15,
    tiles=None,
    scrollWheelZoom=False
)

# 4) Add Mapbox tiles (256px, no @2x), tell Folium tile_size=256
mapbox_tiles = (
    "https://api.mapbox.com/styles/v1/mapbox/streets-v11/"
    "tiles/256/{z}/{x}/{y}?access_token=" + map_box_api_key
)
folium.TileLayer(
    tiles=mapbox_tiles,
    attr='Mapbox',
    name='Mapbox Streets',
    overlay=False,
    control=True,
    tile_size=256,
    zoom_offset=0,    # standard tiles, no retina offset
    min_zoom=1,
    max_zoom=18
).add_to(m)

# Optional: allow switching layers
folium.LayerControl().add_to(m)

# 5) Add title
title_html = """
<h3 align="center" style="font-size:16px">
  <b>Residential Sales in the Mar-a-Lago Security Zone</b>
</h3>
"""
m.get_root().html.add_child(folium.Element(title_html))

# 6) Plot markers
for i, row in df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        tooltip=tooltip_html(i),
        radius=7,
        color='red',
        fill=True
    ).add_to(m)

# 7) Render or save
m  # renders inline in a notebook; otherwise m.save('map.html')


In [10]:
m.save("index.html")

In [11]:
import os

base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SecurityZoneSales
